# Plots for the 2MDM analysis

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from auxPlots import columnsDict,defaultPars
import itertools

pd.option_context('display.max_columns', None)
pd.option_context('display.max_rows', None)

pd.options.mode.chained_assignment = None #Disable copy warnings

plt.rcParams.update(defaultPars)
cm = plt.colormaps['RdYlBu']

#### Get data in pcl format and print all columns names

In [2]:
#Merge with SModelS DataFrame
pclFiles = ['../data/smodels_results.pcl']
dfs = [pd.read_pickle(i) for i in pclFiles]
dataDF = pd.concat(dfs, ignore_index=True)
dataDF.rename(columns=columnsDict,inplace=True)
dropCols = [c for c in dataDF.columns if ('Output' in c or 'weight' in c)]
dataDF.drop(columns=dropCols,inplace=True)
dataDF.set_index('filename',drop=True,inplace=True)
print(dataDF.columns.values.tolist()) #Print all columns names

['mChi', 'mS', 'mZp', 'mass.9000002', 'mass.9000003', 'mass.9000004', 'mass.9000005', 'width.9000002', 'width.9000003', 'width.9000004', 'width.9000005', 'width.9000006', 'width.9900026', 'width.9900032', 'gchi', 'gq', 'sina', 'extpar.4', 'xsec8TeV(fb).9900032', 'xsec8TeV(fb).9900026', 'xsec13TeV(fb).9900032', 'xsec13TeV(fb).9900026', 'BRs.sd.W-,W+', 'BRs.sd.higgs,higgs', 'BRs.sd.Z,Z', 'BRs.sd.t-,t+', 'BRs.sd.chi,chi', 'BRs.sd.ta+,ta-', 'BRs.zp.chi,chi', 'BRs.zp.c,c', 'BRs.zp.q,q', 'BRs.zp.b,b', 'BRs.zp.t-,t+', 'totalxsec13TeV(fb)', 'totalxsec8TeV(fb)', 'CombinedRes', 'Total xsec for missing topologies (fb)', 'Total xsec for missing topologies with displaced decays (fb)', 'Total xsec for missing topologies with prompt decays (fb)', 'Total xsec for topologies outside the grid (fb)', 'ExptRes.result0.maxcond', 'ExptRes.result0.theory prediction (fb)', 'ExptRes.result0.upper limit (fb)', 'ExptRes.result0.expected upper limit (fb)', 'ExptRes.result0.TxNames', 'ExptRes.result0.Mass (GeV)', 

#### Set r = 0 for points with no results 

In [3]:
#Set points without results with r == 0
dataDF.fillna(value={c : 0 for c in dataDF.columns if c.endswith('.r')}, inplace=True)
#Sort points according to r value:
dataDF = dataDF.sort_values(by=['ExptRes.result0.r'],ascending=False)

#### Filter couplings

In [4]:
mZp = 2000.0
mChi = 65.0
gchi = np.sqrt(2)
gq = 0.25
dataDF = dataDF[(np.abs(dataDF['gchi']-gchi) < 0.001) & (dataDF['gq'] == gq) & (dataDF['mChi'] == mChi) & (dataDF['mZp'] == mZp)]
len(dataDF)

3000

### Split dataframe by analysis ID

In [5]:
# Get ExptRes columns and group according to ExptRes attribute
expResCols = np.array(sorted([c for c in dataDF.columns if 'ExptRes.' in c],key = lambda c: c.split('.')[-1]))
expCols = itertools.groupby(expResCols, key = lambda c: c.split('.')[-1])

# Get all analyses
idCols = np.array([c for c in dataDF.columns if 'AnalysisID' in c])
analysis = sorted(dataDF[idCols].stack().drop_duplicates().tolist())
# Get all Txnames
txCols = np.array([c for c in dataDF.columns if 'TxNames' in c and not 'weights' in c])
txnames = sorted(dataDF[txCols].stack().drop_duplicates().tolist())

# Merge all common ExptRes columns into single columns containing a list of results
newDataDF = dataDF.copy()
for key,group in expCols:
    g = list(group)
    newCol = f'ExptRes.result.{key}'
    newDataDF[newCol] = newDataDF[g].values.tolist()
    newDataDF.drop(columns=g,inplace=True)


# Now explode each list of values into distinct rows
expResColsNew = np.array(sorted([c for c in newDataDF.columns if 'ExptRes.' in c],key = lambda c: c.split('.')[-1]))
newDataDFexp = newDataDF.explode(list(expResColsNew))

# Finally group using analsysis IDs
anaGroups = newDataDFexp.groupby(by=['ExptRes.result.AnalysisID'])

In [6]:
# Check if indeed only TRS1 is being used:
print(txnames)

[['TRS1']]


In [7]:
anaDict = {ana[0] : anadf for ana,anadf in anaGroups}

In [8]:
metAna = ['CMS-EXO-20-004']
metDict = {ana : anadf for ana,anadf in anaDict.items() if ana in metAna}

In [9]:
metList = [df[['ExptRes.result.r']] for df in metDict.values()]
BRList = [df[['BRs.sd.chi,chi']] for df in metDict.values()]
r = pd.concat(metList)
br = pd.concat(BRList)

In [10]:
rbrDF = pd.concat([r,dataDF],axis=1)

In [11]:
rbrDF = rbrDF.dropna(subset=['ExptRes.result.r', 'BRs.sd.chi,chi'])

In [12]:
cm = plt.colormaps.get_cmap('Spectral_r')


x = rbrDF['mS']
y = rbrDF['sina']
c = np.array(rbrDF['ExptRes.result.r']) / np.array(rbrDF['BRs.sd.chi,chi'])

plt.hlines(0.26, 200, 1000, linestyle='dashdot', color='black')

im = plt.scatter(x,y,c=c,cmap=cm, vmin=0., vmax=0.4)
cb = plt.colorbar(im)
cb.set_label(label=r'$r_{obs}$')

plt.ylim(0,0.6)
plt.xlim(200,1000)
    
plt.xlabel(r'$m_{S}$ (GeV)')
plt.ylabel(r'$\sin(\alpha$)')
    
plt.text(350., 0.29, r'limit from Higgs signal strength', 
         color='black', backgroundcolor='white')

plt.title(r'SModelS v3.0.0', loc='left')
plt.title(r'2MDM model', loc='right')

plt.tight_layout()
plt.savefig('./figures/2mdm_rs.png')

plt.show()